### Lanczos Method

In [13]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import time

In [67]:
def createBasis(N):
    return np.array(range(2**N))

def spin_x(state,index):
    return state ^ (1<<index)

def spin_z(state,index):
    return 1 if (state & (1<<index)) != 0 else -1

def spin_z_z1_open(state,N):
    spins = np.array([1 if (state & (1<<index)) != 0 else -1 for index in range(N)])[::-1]
    return sum(np.multiply(spins[:-1],spins[1:]))

def hamiltonian_openBoundry(J = 1,g = 0.5,N = 3):
    
    states = createBasis(N)
    
    H = sp.sparse.lil_matrix((2**N,2**N))

    for state in states:
        
        H[state,state] = -J*spin_z_z1_open(state,N)
        
        sx = [spin_x(state,s) for s in range(N)]
        for i in sx:
            H[state,i] = -g
    return H

def lanczosHamiltonian(H,N,L = 6):

    norm = lambda x: np.sqrt(np.sum(np.square(x)))
    
    #L should not be more than basis size
    L = min(2**N,L)

    #Generating Hamiltonian
    #H = hamiltonian_openBoundry(J,g,N)
    
    #Generating a Random Normalised Vector - psi will be our Krylov Space
    psi = np.array([np.random.uniform(size = 2**N)])
    #psi = np.array(np.ones(2**N))
    psi = [psi.T/norm(psi)]
    
    #List of Diagonal and Off-diagonal terms
    diag,offdiag = [0],[0]

    #First Iteration
    
    psi.append(H@psi[0]) #Applying Hamiltonian
    
    diag.append(psi[0].T@psi[1]) #Finding Diagonal Term
    
    psi[1] -= diag[1]*psi[0] #New vector must be Orthogonal
    # offdiag.append(norm(psi[1])) #Offdiagonal term is the Norm of the next Vector
    
    for i in range(2,L+1):
        
        offdiag.append(norm(psi[i-1])) #Offdiagonal term is the Norm of the current Vector
        assert offdiag[i-1] != 0 ,"Offdiagonal term is zero"
        psi[i-1] /= offdiag[i-1] #Normalising the Orthogonal Vector
        
        psi.append(H@psi[i-1]) #New vector which is orthogonal but not normal

        diag.append(psi[i-1].T @ psi[i]) 

        psi[i] = psi[i] - diag[i-1]*psi[i-1] - offdiag[i-2]*psi[i-2] 

        psi[i] /= norm(psi[i])

    diag = np.array(diag[1:]).flatten()
    offdiag = offdiag[1:]
    
    #print(diag)
    return sp.sparse.diags([offdiag,diag,offdiag],offsets = [-1,0,1]),diag,offdiag

"""
psi next = h*psi_prev - prev diag * psi_prev - prev_offdiag*psi_prevprev
diag = psi_prev*H*psi_prev
offdiag = norm of complete psi
""";

In [78]:
J,g,N,L = 10,2,12,20

start = time.time()
H = hamiltonian_openBoundry(J,g,N)
t1 = time.time()
Hk,diag,offdiag = lanczosHamiltonian(H,N,L)
t2 = time.time()

t2 = t2 - t1
t1 = t1 - start

Hk.asformat("array"),Hk.shape;

gs,gs_vec = sp.sparse.linalg.eigsh(H,k = 1,which="SA")
gsk = sp.linalg.eigvalsh_tridiagonal(diag,offdiag)

data = {
    "J" : J,
    "g" : g,
    "N" : N,
    "L" : L,
    "Regular GSE": f'{gs[0]:.3f} ; {t1:.3f} sec',
    "Lanczos GSE": f'{min(gsk):.3f} ; {t2:.3f} sec' ,
    "Error": f'{(abs(gs-min(gsk))[0]) :.3f}',
    "Error %": f'{(abs(gs-min(gsk))*100/abs(gs))[0]:.2f} %',
    "Time Gain(s)": f'{(t1-t2):.4f} sec'
}
print(*[f'{k} = {v}' for k,v in data.items()],sep = "\n")

J = 10
g = 2
N = 12
L = 20
Regular GSE = -111.404 ; 0.214 sec
Lanczos GSE = -110.144 ; 0.032 sec
Error = 1.260
Error % = 1.13 %
Time Gain(s) = 0.1815 sec


In [61]:
# CHAT GPT while testing
import numpy as np
import scipy as sp

def lanczosHamiltonian_GPT(H, L=6):
    """
    Generate a tridiagonal (Krylov-space) Hamiltonian using the Lanczos algorithm.
    """
    N = int(np.log2(H.shape[0]))  # infer system size if needed
    L = min(2**N, L)
    
    norm = lambda x: np.sqrt(np.sum(np.square(np.abs(x))))
    
    # Random normalized starting vector
    psi0 = np.random.randn(H.shape[0]) + 1j*np.random.randn(H.shape[0])
    psi0 /= norm(psi0)
    
    diag = []
    offdiag = []
    
    psi_prev = np.zeros_like(psi0)
    psi = psi0.copy()
    
    for i in range(L):
        phi = H @ psi
        alpha = np.vdot(psi, phi).real
        diag.append(alpha)
        
        if i > 0:
            phi -= alpha * psi + beta * psi_prev
        else:
            phi -= alpha * psi
        
        beta = norm(phi)
        if beta < 1e-12:
            break
        
        offdiag.append(beta)
        psi_prev = psi
        psi = phi / beta
    
    # Construct tridiagonal sparse matrix
    T = sp.sparse.diags(
        [offdiag, diag, offdiag],
        offsets=[-1, 0, 1],
        format='csr'
    )
    
    return T, np.array(diag), np.array(offdiag)

# Example: 4x4 random Hermitian matrix
#H = np.random.randn(4,4) + 1j*np.random.randn(4,4)
#H = (H + H.conj().T)/2  # make it Hermitian
#
#T, diag, offdiag = lanczosHamiltonian(H, L=4)
#
#print("Tridiagonal matrix:\n", T.toarray())
#print("Diagonal elements:", diag)
#print("Off-diagonal elements:", offdiag)

J,g,N,L = 10,0,13,15


H = hamiltonian_openBoundry(J,g,N)

Hk,diag,offdiag = lanczosHamiltonian_GPT(H,L)


gs,gs_vec = sp.sparse.linalg.eigsh(H,k = 1,which="SA")
gsk = sp.linalg.eigvalsh_tridiagonal(diag,offdiag)

data = {
    "Regular GSE": f'{gs[0]:.3f}',
    "Lanczos GSE": f'{min(gsk):.3f}' ,
    "Error": f'{(abs(gs-min(gsk))[0]) :.3f}',
    "Error %": f'{(abs(gs-min(gsk))*100/abs(gs))[0]:.2f} %',
}
print(*[f'{k} = {v}' for k,v in data.items()],sep = "\n")

Regular GSE = -120.000
Lanczos GSE = -120.000
Error = 0.000
Error % = 0.00 %
